In [1]:
import pandas as pd
import numpy as np

import numba

import bebi103

import altair as alt
import altair_catplot as altcat

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
df = pd.read_csv('../data/hw_4.2_caulobacter_growth_image_processing_results.csv')

df.head()

,time (min),area (sq um),growth_event,bacterium
0,1.0,1.300624,0,1
1,2.0,1.314144,0,1
2,3.0,1.295216,0,1
3,4.0,1.314144,0,1
4,5.0,1.341184,0,1


Since the length of division is different for all growth events we should add another column that restarts the time count from 0 every time there's a division.

In [3]:
time = []
j = 0
for i in df['growth_event'].diff():
    if i == 0:
        j += 1
        time.append(j)
    else:
        j = 0
        time.append(j)

df['new time'] = time

In [4]:
df.head()

,time (min),area (sq um),growth_event,bacterium,new time
0,1.0,1.300624,0,1,0
1,2.0,1.314144,0,1,1
2,3.0,1.295216,0,1,2
3,4.0,1.314144,0,1,3
4,5.0,1.341184,0,1,4


Let's start with a subset of the data. We will look at bacterium 1.

In [5]:
df_bacterium1 = df.loc[df['bacterium'] == 1]

In [6]:
p = bokeh.plotting.figure(plot_width=650,
                          plot_height=250,
                          x_axis_label='time (min)',
                          y_axis_label='cell area (sq µm)')

# Specify the glyphs
colors = ['#1f78b4', '#a6cee3']
for i, g in df_bacterium1.groupby('growth_event'):
    p.circle(g['time (min)'], g['area (sq um)'], size=3, color=colors[i%2])

bokeh.io.show(p)

#### No hierarchy

Let's just model a single growth event for now with no hierarchy using the exponential model.

In [7]:
def data_prior_pred(t):
    '''
    Samples parameter values according to the prior and generates
    data y at the values given in t.
    '''
    # Sample parameter values according to priors
    a = np.random.normal(1.2, 0.4)
    k = np.random.normal(0.01, 0.003)
    sigma = np.abs(np.random.normal(0, 0.1))
    
    # Generate random data according to the likelihood
    return np.random.normal(a * np.exp(k * t), sigma)

In [8]:
p = bokeh.plotting.figure(height=300, width=450,
                          x_axis_label='time',
                          y_axis_label='area')

t = df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 'new time'].values

# Plot simulated data
for i in range(100):
    p.circle(t, data_prior_pred(t), size=3, alpha=0.1)

# Plot original data
p.circle(t, df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 'area (sq um)'].values, 
         color='black', size=4)
bokeh.io.show(p)

In [9]:
p = bokeh.plotting.figure(height=300, width=450,
                          x_axis_label='time',
                          y_axis_label='area')

t = df_bacterium1.loc[df_bacterium1['growth_event'] == 3, 'new time'].values

# Plot simulated data
for i in range(100):
    p.circle(t, data_prior_pred(t), size=3, alpha=0.1)

# Plot original data
p.circle(t, df_bacterium1.loc[df_bacterium1['growth_event'] == 3, 'area (sq um)'].values, 
         color='black', size=4)
bokeh.io.show(p)

Now let's try the linear model.

In [10]:
def data_prior_pred_linear(t):
    '''
    Samples parameter values according to the prior and generates
    data y at the values given in t.
    '''
    # Sample parameter values according to priors
    a = np.random.normal(1.2, 0.4)
    b = np.random.normal(0.01, 0.003)
    sigma = np.abs(np.random.normal(0, 0.1))
    
    # Generate random data according to the likelihood
    return np.random.normal(a + b * t, sigma)

In [11]:
p = bokeh.plotting.figure(height=300, width=450,
                          x_axis_label='time',
                          y_axis_label='area')

t = df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 'new time'].values

# Plot simulated data
for i in range(100):
    p.circle(t, data_prior_pred_linear(t), size=3, alpha=0.1)

# Plot original data
p.circle(t, df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 'area (sq um)'].values, 
         color='black', size=4)
bokeh.io.show(p)

#### One level hierarchical model

Let's try a one level hierarchical model for bacterium 1.

In [18]:
model_code_pri_pred = """
data {
  // Number of data points for each experiment
  int N;
  // Number of entries in each level of the hierarchy
  int J_1;
  // Input of parameters of the priors
  real hyper_a0_mu;
  real hyper_a0_sigma;
  
  real hyper_k0_mu;
  real hyper_k0_sigma;
  
  real hyper_a0_tau;
  real hyper_k0_tau;
}


generated quantities {
  // Total number of data points 
  real a[N * J_1];
  real k[N * J_1];
  
  // Priors
  real a0 = normal_rng(hyper_a0_mu, hyper_a0_sigma);
  real k0 = normal_rng(hyper_k0_mu, hyper_k0_sigma);
  
  real a0_sigma = fabs(normal_rng(0, hyper_a0_sigma));
  real k0_sigma = fabs(normal_rng(0, hyper_k0_sigma));
  
  real a0_tau = fabs(normal_rng(0, hyper_a0_tau));
  real k0_tau = fabs(normal_rng(0, hyper_k0_tau));
  
  // Second layer
  real a_1[J_1]; 
  real k_1[J_1];
  
  for (i in 1:J_1) {
    a_1[i] = normal_rng(a0, a0_tau);
    k_1[i] = normal_rng(k0, k0_tau);
    for (j in 1:N) {
      a[(i - 1) * N + j] = normal_rng(a_1[i], a0_sigma);
      k[(i - 1) * N + j] = normal_rng(k_1[i], k0_sigma);
    }
  }
    
  }
"""

In [19]:
# Compile
sm_gen = bebi103.stan.StanModel(model_code=model_code_pri_pred)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_fe32cb4498718f51dde97b497b28c319 NOW.
/Users/madelynw/anaconda3/lib/python3.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /var/folders/cn/x4zmqr494ln7sfp5z2kytzqw0000gp/T/tmps7xwt2t3/stanfit4anon_model_fe32cb4498718f51dde97b497b28c319_4258100889973328742.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [ ]:
# Number of iteration
N_iter = 1000

# 98 data points per experiments, 3 growth events
data = dict(N=98,
            J_1=3,
            hyper_a0_mu=1.4,
            hyper_a0_sigma=0.3,
            hyper_k0_mu=0.01,
            hyper_k0_sigma=0.002,
            hyper_a0_tau=0.1,
            hyper_k0_tau=0.001)

# Sample
df_pred = sm_gen.sampling(data=data,
                     algorithm='Fixed_param',
                     warmup=0,
                     chains=1,
                     iter=N_iter)

df_samples_a = bebi103.stan.extract_array(df_pred, name='a')
df_samples_k = bebi103.stan.extract_array(df_pred, name='k')

p_a = bebi103.viz.predictive_ecdf(df_pred, 
                                'a', 
                                x_axis_label='a')
p_k = bebi103.viz.predictive_ecdf(df_pred, 
                                'k', 
                                x_axis_label='k')
# Plot
bokeh.io.show(bokeh.layouts.gridplot([p_a, p_k], ncols=2))

Check for negative values:

In [ ]:
len(df_samples_a[df_samples_a['a'] < 0]) / len(df_samples_a)

In [ ]:
len(df_samples_k[df_samples_k['k'] < 0]) / len(df_samples_k)

In [ ]:
len(df_samples_a)

In [ ]:
a = df_samples_a['a'].values
k = df_samples_k['k'].values

p = bokeh.plotting.figure(height=300, width=450,
                          x_axis_label='time',
                          y_axis_label='area')

t = df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 'new time'].values

# Plot simulated data
for i in range(0, len(df_samples_a), 1000):
    p.circle(t, a[i] * np.exp(k[i] * t), size=3, alpha=0.1)

# Plot original data
p.circle(t, df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 'area (sq um)'].values, 
         color='black', size=4)
bokeh.io.show(p)

Trying the same priors/samples for the linear model.

In [56]:
a = df_samples_a['a'].values
k = df_samples_k['k'].values

p = bokeh.plotting.figure(height=300, width=450,
                          x_axis_label='time',
                          y_axis_label='area')

t = df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 'new time'].values

# Plot simulated data
for i in range(0, len(df_samples_a), 1000):
    p.circle(t, a[i] + k[i] * t, size=3, alpha=0.1)

# Plot original data
p.circle(t, df_bacterium1.loc[df_bacterium1['growth_event'] == 0, 'area (sq um)'].values, 
         color='black', size=4)
bokeh.io.show(p)